In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
pd.options.mode.chained_assignment = None

**Import Train Data**

In [2]:
train = pd.read_csv(r"...\titanic\train.csv")

In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


**Check null percentage (higher = unused)**

In [4]:
miss = train.isnull().sum()*100/len(train)
miss_df =pd.DataFrame({'Collumn':train.columns,'Miss':miss})

In [5]:
miss_df

,Collumn,Miss
PassengerId,PassengerId,0.000000
Survived,Survived,0.000000
Pclass,Pclass,0.000000
Name,Name,0.000000
Sex,Sex,0.000000
Age,Age,19.865320
SibSp,SibSp,0.000000
Parch,Parch,0.000000
Ticket,Ticket,0.000000
Fare,Fare,0.000000


**column Cabin is droped**

In [6]:
train = train.drop(columns=['Cabin','Ticket','Name','PassengerId'])

In [7]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


**to fill the N/A get information about age distribution among sex**

In [8]:
age = train[['Survived','Sex','Age']].dropna()

In [9]:
age.groupby(['Sex','Survived']).describe()

Age                                                      
                 count       mean        std   min    25%   50%    75%   max
Sex    Survived                                                             
female 0          64.0  25.046875  13.618591  2.00  16.75  24.5  33.25  57.0
       1         197.0  28.847716  14.175073  0.75  19.00  28.0  38.00  63.0
male   0         360.0  31.618056  14.056019  1.00  21.75  29.0  39.25  74.0
       1          93.0  27.276022  16.504803  0.42  18.00  28.0  36.00  80.0

**fill empty with the median of age based on the sex and survavibility**

In [10]:
train.loc[((train["Sex"]=="male") & (train["Survived"] == 0)& np.isnan(train["Age"])),'Age']=29
train.loc[((train["Sex"]=="male") & (train["Survived"] == 1)& np.isnan(train["Age"])),'Age']=28
train.loc[((train["Sex"]=="female") & (train["Survived"] == 0)& np.isnan(train["Age"])),'Age']=24.5
train.loc[((train["Sex"]=="female") & (train["Survived"] == 1)& np.isnan(train["Age"])),'Age']=28

In [11]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,24.5,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [12]:
train.groupby(['Embarked'],dropna=False).describe()

Survived                                              Pclass  \
            count      mean       std  min  25%  50%  75%  max  count   
Embarked                                                                
C           168.0  0.553571  0.498608  0.0  0.0  1.0  1.0  1.0  168.0   
Q            77.0  0.389610  0.490860  0.0  0.0  0.0  1.0  1.0   77.0   
S           644.0  0.336957  0.473037  0.0  0.0  0.0  1.0  1.0  644.0   
NaN           2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0    2.0   

                    ... Parch        Fare                                 \
              mean  ...   75%  max  count       mean        std      min   
Embarked            ...                                                    
C         1.886905  ...   1.0  3.0  168.0  59.954144  83.912994   4.0125   
Q         2.909091  ...   0.0  5.0   77.0  13.276030  14.188047   6.7500   
S         2.350932  ...   0.0  6.0  644.0  27.079812  35.887993   0.0000   
NaN       1.000000  ...   0.0  0.0    2.0  80.000000   0.000000  80.0000   

                                                
               25%    50%        75%       max  
Embarked                                        
C         13.69795  29.70  78.500025  512.3292  
Q          7.75000   7.75  15.500000   90.0000  
S          8.05000  13.00  27.900000  263.0000  
NaN       80.00000  80.00  80.000000   80.0000  

[4 rows x 48 columns]

In [13]:
train.loc[pd.isnull(train['Embarked']),'Embarked']='U'

In [14]:
train.groupby(['Embarked'],dropna=False).describe()

Survived                                              Pclass  \
            count      mean       std  min  25%  50%  75%  max  count   
Embarked                                                                
C           168.0  0.553571  0.498608  0.0  0.0  1.0  1.0  1.0  168.0   
Q            77.0  0.389610  0.490860  0.0  0.0  0.0  1.0  1.0   77.0   
S           644.0  0.336957  0.473037  0.0  0.0  0.0  1.0  1.0  644.0   
U             2.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0    2.0   

                    ... Parch        Fare                                 \
              mean  ...   75%  max  count       mean        std      min   
Embarked            ...                                                    
C         1.886905  ...   1.0  3.0  168.0  59.954144  83.912994   4.0125   
Q         2.909091  ...   0.0  5.0   77.0  13.276030  14.188047   6.7500   
S         2.350932  ...   0.0  6.0  644.0  27.079812  35.887993   0.0000   
U         1.000000  ...   0.0  0.0    2.0  80.000000   0.000000  80.0000   

                                                
               25%    50%        75%       max  
Embarked                                        
C         13.69795  29.70  78.500025  512.3292  
Q          7.75000   7.75  15.500000   90.0000  
S          8.05000  13.00  27.900000  263.0000  
U         80.00000  80.00  80.000000   80.0000  

[4 rows x 48 columns]

In [15]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,24.5,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


Change all string data into numeric category

In [16]:
train.Sex = pd.Categorical(train.Sex)
train['Sex'] = train.Sex.cat.codes

In [17]:
train.Embarked = pd.Categorical(train.Embarked)
train['Embarked'] = train.Embarked.cat.codes

In [18]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,2
887,1,1,0,19.0,0,0,30.0000,2
888,0,3,0,24.5,1,2,23.4500,2
889,1,1,1,26.0,0,0,30.0000,0


Time to build the model

In [19]:
clf = DecisionTreeClassifier(random_state=0)
Y = train.pop('Survived')
X = train

In [20]:
cross_val_score(clf, X, Y, cv=5)

array([0.72625698, 0.80898876, 0.82022472, 0.76404494, 0.8258427 ])

In [21]:
test = pd.read_csv(r"...\titanic\test.csv")

In [22]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [23]:
miss = test.isnull().sum()*100/len(test)
miss_df =pd.DataFrame({'Collumn':test.columns,'Miss':miss})

In [24]:
miss_df

,Collumn,Miss
PassengerId,PassengerId,0.000000
Pclass,Pclass,0.000000
Name,Name,0.000000
Sex,Sex,0.000000
Age,Age,20.574163
SibSp,SibSp,0.000000
Parch,Parch,0.000000
Ticket,Ticket,0.000000
Fare,Fare,0.239234
Cabin,Cabin,78.229665


In [25]:
test.groupby(['Sex']).describe()

PassengerId                                                           \
             count         mean         std    min     25%     50%      75%   
Sex                                                                           
female       152.0  1096.789474  123.345508  893.0  989.50  1093.5  1205.25   
male         266.0  1102.620301  119.520499  892.0  999.25  1103.5  1203.75   

               Pclass            ... Parch        Fare                        \
           max  count      mean  ...   75%  max  count       mean        std   
Sex                              ...                                           
female  1306.0  152.0  2.144737  ...   1.0  9.0  152.0  49.747699  73.108716   
male    1309.0  266.0  2.334586  ...   0.0  9.0  265.0  27.527877  41.079423   

                                                   
         min      25%      50%      75%       max  
Sex                                                
female  6.95  8.62605  21.5125  55.4417  512.3292  
male    0.00  7.85420  13.0000  26.5500  262.3750  

[2 rows x 48 columns]

In [26]:
pax_id  = test['PassengerId'].values
test = test.drop(columns=['Cabin','Ticket','Name','PassengerId'])
test.loc[((test["Sex"]=="male") & np.isnan(test["Age"])),'Age']=30.27
test.loc[((test["Sex"]=="female")& np.isnan(test["Age"])),'Age']=30.27
test.loc[((test["Sex"]=="male") & np.isnan(test["Fare"])),'Fare']=21.5125
test.loc[((test["Sex"]=="female")& np.isnan(test["Fare"])),'Fare']=13
test.loc[pd.isnull(test['Embarked']),'Embarked']='U'
test.Sex = pd.Categorical(test.Sex)
test['Sex'] = test.Sex.cat.codes
test.Embarked = pd.Categorical(test.Embarked)
test.loc
test['Embarked'] = test.Embarked.cat.codes

In [27]:
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.50,0,0,7.8292,1
1,3,0,47.00,1,0,7.0000,2
2,2,1,62.00,0,0,9.6875,1
3,3,1,27.00,0,0,8.6625,2
4,3,0,22.00,1,1,12.2875,2
...,...,...,...,...,...,...,...
413,3,1,30.27,0,0,8.0500,2
414,1,0,39.00,0,0,108.9000,0
415,3,1,38.50,0,0,7.2500,2
416,3,1,30.27,0,0,8.0500,2


In [28]:
model = clf.fit(X,Y)

In [29]:
X_test = test.values

In [30]:
result = model.predict(X_test)

In [32]:
result_df = pd.DataFrame({'PassengerId':pax_id,"Survived":result.tolist()})

In [33]:
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [34]:
result_df.to_csv(r"...\submission.csv",index=False)

ATTEMPT 2: Using Random Forest

In [35]:
clf = RandomForestClassifier(max_depth=100,criterion='entropy',random_state=0)

In [36]:
clf.fit(X,Y)

RandomForestClassifier(criterion='entropy', max_depth=100, random_state=0)

In [37]:
result = clf.predict(X_test).tolist()

In [38]:
result_df = pd.DataFrame({'PassengerId':pax_id,"Survived":result})

In [39]:
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [40]:
result_df.to_csv(r"...\titanic\submission.csv",index=False)